In [1]:
import pandas as pd
import os

In [2]:
data = pd.read_csv('./SAheart.csv', sep=',', header=0)
data = data.drop(columns=['row.names'])
data.head(10)

,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
0,160,12.00,5.73,23.11,Present,49,25.30,97.20,52,1
1,144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1
2,118,0.08,3.48,32.28,Present,52,29.14,3.81,46,0
3,170,7.50,6.41,38.03,Present,51,31.99,24.26,58,1
4,134,13.60,3.50,27.78,Present,60,25.99,57.34,49,1
5,132,6.20,6.47,36.21,Present,62,30.77,14.14,45,0
6,142,4.05,3.38,16.20,Absent,59,20.81,2.62,38,0
7,114,4.08,4.59,14.60,Present,62,23.11,6.72,58,1
8,114,0.00,3.83,19.40,Present,49,24.86,2.49,29,0
9,132,0.00,5.80,30.96,Present,69,30.11,0.00,53,1


In [3]:
data1 = data.copy()
data1['famhist'] = (data['famhist']=='Present').astype(int)
data1.head(10)

,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
0,160,12.00,5.73,23.11,1,49,25.30,97.20,52,1
1,144,0.01,4.41,28.61,0,55,28.87,2.06,63,1
2,118,0.08,3.48,32.28,1,52,29.14,3.81,46,0
3,170,7.50,6.41,38.03,1,51,31.99,24.26,58,1
4,134,13.60,3.50,27.78,1,60,25.99,57.34,49,1
5,132,6.20,6.47,36.21,1,62,30.77,14.14,45,0
6,142,4.05,3.38,16.20,0,59,20.81,2.62,38,0
7,114,4.08,4.59,14.60,1,62,23.11,6.72,58,1
8,114,0.00,3.83,19.40,1,49,24.86,2.49,29,0
9,132,0.00,5.80,30.96,1,69,30.11,0.00,53,1


In [4]:
y = data1['chd'].copy()
X = data1.copy().drop(columns=['chd'])
print(X.head())
print(y.head())
print(X.shape)
print(y.shape)

   sbp  tobacco   ldl  adiposity  famhist  typea  obesity  alcohol  age
0  160    12.00  5.73      23.11        1     49    25.30    97.20   52
1  144     0.01  4.41      28.61        0     55    28.87     2.06   63
2  118     0.08  3.48      32.28        1     52    29.14     3.81   46
3  170     7.50  6.41      38.03        1     51    31.99    24.26   58
4  134    13.60  3.50      27.78        1     60    25.99    57.34   49
0    1
1    1
2    0
3    1
4    1
Name: chd, dtype: int64
(462, 9)
(462,)


In [5]:
# apply normalization
from sklearn.preprocessing import StandardScaler as SS
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer(
    [("ss", SS(), ["sbp", "tobacco", "ldl", "adiposity", "obesity", "alcohol"])],
    remainder= "passthrough",
)

In [7]:
xx = ct.fit_transform(X)
cols = [j.split('__')[-1] for j in ct.get_feature_names_out()]
print(cols)
X1 = pd.DataFrame(xx,columns = cols)
print(X1.shape)
X1.head()

['sbp', 'tobacco', 'ldl', 'adiposity', 'obesity', 'alcohol', 'famhist', 'typea', 'age']
(462, 9)


,sbp,tobacco,ldl,adiposity,obesity,alcohol,famhist,typea,age
0,1.058564,1.823073,0.478412,-0.295503,-0.176786,3.277738,1.0,49.0,52.0
1,0.277089,-0.790237,-0.159680,0.412140,0.671373,-0.612745,0.0,55.0,63.0
2,-0.992806,-0.774980,-0.609245,0.884332,0.735519,-0.541183,1.0,52.0,46.0
3,1.546985,0.842264,0.807126,1.624141,1.412621,0.295062,1.0,51.0,58.0
4,-0.211332,2.171805,-0.599577,0.305351,-0.012856,1.647775,1.0,60.0,49.0


In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [9]:
train_X, test_X, train_y, test_y = train_test_split(X1,y,test_size=0.1, random_state=1234)

In [10]:
len(train_y)

415

In [11]:
LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr', max_iter=1000).fit(train_X,train_y)
# LR = LinearRegression().fit(train_X,train_y)

In [12]:
yhat = LR.predict(test_X)
print(yhat)
print(test_y)
print(LR.score(test_X, test_y))
confusion_matrix(test_y, yhat)

[1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 1 1 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 1 1
 0 0 0 0 0 0 0 0 0 1]
59     0
138    0
262    0
248    0
302    0
402    1
319    0
362    0
220    0
254    0
429    0
444    0
283    0
205    0
214    0
92     0
312    1
111    1
22     0
189    1
64     0
27     1
296    0
40     1
408    0
265    0
445    0
271    1
304    0
7      1
315    0
29     1
353    1
392    0
252    1
306    0
207    0
141    1
285    0
133    0
330    0
71     0
441    0
378    0
366    0
369    0
364    1
Name: chd, dtype: int64
0.7659574468085106


array([[29,  5],
       [ 6,  7]], dtype=int64)

In [13]:
from sklearn.neural_network import MLPClassifier as MLP

In [14]:
cls = MLP(random_state=0, max_iter=100, hidden_layer_sizes=(32,64,32))

In [15]:
cls.fit(train_X, train_y)

MLPClassifier(hidden_layer_sizes=(32, 64, 32), max_iter=100, random_state=0)

In [16]:
yhat = cls.predict(test_X)

In [17]:
print(yhat)
print(test_y)
print(cls.score(test_X, test_y))
confusion_matrix(test_y, yhat)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0]
59     0
138    0
262    0
248    0
302    0
402    1
319    0
362    0
220    0
254    0
429    0
444    0
283    0
205    0
214    0
92     0
312    1
111    1
22     0
189    1
64     0
27     1
296    0
40     1
408    0
265    0
445    0
271    1
304    0
7      1
315    0
29     1
353    1
392    0
252    1
306    0
207    0
141    1
285    0
133    0
330    0
71     0
441    0
378    0
366    0
369    0
364    1
Name: chd, dtype: int64
0.723404255319149


array([[33,  1],
       [12,  1]], dtype=int64)

In [18]:
NN = MLP(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
NN.fit(train_X, train_y)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 2), random_state=1,
              solver='lbfgs')

In [19]:
yhat = NN.predict(test_X)
print(yhat)
print(test_y)

confusion_matrix(test_y, yhat)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0]
59     0
138    0
262    0
248    0
302    0
402    1
319    0
362    0
220    0
254    0
429    0
444    0
283    0
205    0
214    0
92     0
312    1
111    1
22     0
189    1
64     0
27     1
296    0
40     1
408    0
265    0
445    0
271    1
304    0
7      1
315    0
29     1
353    1
392    0
252    1
306    0
207    0
141    1
285    0
133    0
330    0
71     0
441    0
378    0
366    0
369    0
364    1
Name: chd, dtype: int64


array([[34,  0],
       [13,  0]], dtype=int64)

In [20]:
round(NN.score(test_X,test_y), 4)

0.7234

In [58]:
x = train_X.to_numpy()
y = train_y.to_numpy()

In [59]:
import torch
from torch import nn
from torch.utils.data import Dataset

ModuleNotFoundError: No module named 'torch'